In [1]:
#importing dependecies
#import praw  #not being used
import pandas as pd
#import config #not being used
from psaw import PushshiftAPI 
import datetime as dt
import re

In [2]:
api = PushshiftAPI()

In [3]:
def ticker_reddit_scraper(ticker,year,month,day):
    #querie reddit for ticker
    gen = api.search_comments(q=ticker, 
                              subreddit='wallstreetbets',
                              after=int(dt.datetime(year, month, day).timestamp()),
                              before=int(dt.datetime(year, month, day+1).timestamp()),
                              filter=['body','author','subreddit']
                             )
    #max responses retrieved
    max_response=10000
    cache=[]
    for i in gen:
        #take only the dictionary from queries
        cache.append(i[-1])
        if len(cache) >= max_response:
            break
    df = pd.DataFrame(cache)
    if df.empty != True:
        try:
            df['created_utc']=pd.to_datetime(df['created_utc'],unit='s').dt.date
        except KeyError:
            return df
        df = df.drop(columns=['created'])
        df.rename(columns={'author':'username','created_utc':'date'}, inplace=True)
        df=df[['date','body','username','subreddit']]
    return df

In [4]:
tickers=['BB', 'CRSR', 'TG', 'WEBR', 'ABNB', 'OI', 'CHGG', 'INTC', 'HP', 'SQ',
         'ROKU', 'PT', 'VXX', 'BYND', 'JBLU', 'GBTC',
         'MS', 'DKNG', 'FCF', 'GE', 'BJ', 'LCID',  'CS', 'KO',
         'WTI', 'RIG', 'RC', 'BA', 'MMAT', 'RKLB', 'UNH', 'RE', 
         'CSCO', 'VC', 'ZM', 'TIL', 'IBKR', 'CRM', 'IBM', 'ET', 'CP',
         'MSTR', 'DAC', 'DTC', 'NYT', 'TH', 'BP', 'LMT', 'SM', 
         'ITT', 'BBBY', 'TTD', 'PBR', 'SPCE', 'UPST', 'AFRM', 'NG']

In [5]:
word_tickers=['UP','YOU', 'IS', 'IT', 'FOR','HE', 'PLAN','BOIL','POST','EVER',
              'WOW','FANG','EDIT','RIDE','REAL', 'LOW','ANY','DRIP', 'ST', 
              'FL','OPEN','NEXT','HOOD','OUT','ICE','BIG','SNOW','HAS','SHOP', 
              'ALL','AMP','DASH','GO','ON','BE','DNA','NET','AM','CD','PAY',
              'ME','UFO', 'USB','SAVE', 'WE','NOW','OR','COIN','MA','SO','CAN',
              'MAX','HR','WISH']

In [6]:
tickers_done=['GME', 'TSLA', 'AAPL', 'TWTR', 'ATER', 'AMC','AMD', 
         'MSFT', 'HD', 'FB', 'AMZN', 'NU', 'DTE', 'GOOG',
         'NFLX', 'SOFI', 'TA', 'BBIG', 'NVDA', 'AI', 'ZIM','VTNR',
         'PLTR', 'RIVN', 'LUNA', 'TTWO', 'OXY', 'BABA',  'WMT',
         'DWAC', 'CC', 'COO', 'LFG', 'RBLX', 'DIS', 'SE', 'SNDL', 'PARA',
         'SP', 'XOM', 'ES', 'JD', 'PTON', 'WBD', 'TGT', 'OG',
         'DM', 'DOW', 'TLRY', 'NIO', 'PANW', 'CVNA', 'JPM', 'IP', 'TD', 'GS', 
         'CLOV', 'PYPL', 'GM', 'IQ','MCD', 'MULN', 'DE', 'VZ', 'WW', 
         'EA']

In [30]:
years = [2021,2020,2019,2018]
months=[1,2,3,4,5,6,7,8,9,10,11,12]
for ticker in tickers:
    for year in years:
        for month in months:
            for day in months:
                csv = ticker_reddit_scraper(ticker,year,month,day)
                if csv.empty != True:
                    csv.to_csv('csv/'+ticker+'_'+str(year)+'_'+str(month)+'_'+str(day)+'.csv')

C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 524
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 522
  warnings.warn("Got non 200 code %s" % response.status_code)


In [7]:
years = [2021,2020,2019,2018]
months=[1,2,3,4,5,6,7,8,9,10,11,12]
days = [13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]
for ticker in tickers:
    for year in years:
        for month in months:
            for day in days:
                csv = ticker_reddit_scraper(ticker,year,month,day)
                if csv.empty != True:
                    csv.to_csv('csv/'+ticker+'_'+str(year)+'_'+str(month)+'_'+str(day)+'.csv',index=False)

C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 502
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 522
  warnings.warn("Got non 200 code %s" % response.status_code)


In [8]:
def ticker_reddit_scraper_ends(gen):
    
    #max responses retrieved
    max_response=10000
    cache=[]
    for i in gen:
        #take only the dictionary from queries
        cache.append(i[-1])
        if len(cache) >= max_response:
            break
    df = pd.DataFrame(cache)
    if df.empty != True:
        try:
            df['created_utc']=pd.to_datetime(df['created_utc'],unit='s').dt.date
        except KeyError:
            return df
        df = df.drop(columns=['created'])
        df.rename(columns={'author':'username','created_utc':'date'}, inplace=True)
        df=df[['date','body','username','subreddit']]
    return df

In [11]:
years = [2021,2020,2019,2018]
months=[1,2,3,4,5,6,7,8,9,10,11,12]
days = [28,29,30,31]
for ticker in tickers:
    for year in years:
        for month in months:
            for day in days:
                if (month == 2 and day==28)|(month in [4,6,9,11] and day == 30)|(month in [1,3,5,7,8,10] and day == 31):
                    gen = api.search_comments(q=ticker, 
                                  subreddit='wallstreetbets',
                                  after=int(dt.datetime(year, month, day).timestamp()),
                                  before=int(dt.datetime(year, month+1, 1).timestamp()),
                                  filter=['body','author','subreddit']
                                 )
                elif month ==12 and day ==31:
                    gen = api.search_comments(q=ticker, 
                                  subreddit='wallstreetbets',
                                  after=int(dt.datetime(year, month, day).timestamp()),
                                  before=int(dt.datetime(year, 1, 1).timestamp()),
                                  filter=['body','author','subreddit']
                                 )
                else:
                    try:
                        gen = api.search_comments(q=ticker, 
                                  subreddit='wallstreetbets',
                                  after=int(dt.datetime(year, month, day).timestamp()),
                                  before=int(dt.datetime(year, month, day+1).timestamp()),
                                  filter=['body','author','subreddit']
                                 )
                    except:
                        continue
                csv = ticker_reddit_scraper_ends(gen)
                if csv.empty != True:
                    csv.to_csv('csv/'+ticker+'_'+str(year)+'_'+str(month)+'_'+str(day)+'.csv',index=False)#querie reddit for ticker

In [23]:
def ticker_reddit_scraper(ticker,year,month):
    #querie reddit for ticker
    gen = api.search_comments(q=ticker, 
                              subreddit='stocks',
                              after=int(dt.datetime(year, month,1).timestamp()),
                              before=int(dt.datetime(year, month+1,1).timestamp()),
                              filter=['body','author','subreddit']
                             )
    #max responses retrieved
    max_response=310000
    cache=[]
    for i in gen:
        #take only the dictionary from queries
        cache.append(i[-1])
        if len(cache) >= max_response:
            break
    df = pd.DataFrame(cache)
    if df.empty != True:
        try:
            df['created_utc']=pd.to_datetime(df['created_utc'],unit='s').dt.date
        except KeyError:
            return df
        df = df.drop(columns=['created'])
        df.rename(columns={'author':'username','created_utc':'date'}, inplace=True)
        df=df[['date','body','username','subreddit']]
    return df

In [24]:
years = [2021,2020,2019,2018]
months=[1,2,3,4,5,6,7,8,9,10,11]
for ticker in tickers+tickers_done:
    for year in years:
        for month in months:
                csv = ticker_reddit_scraper(ticker,year,month)
                if csv.empty != True:
                    csv.to_csv('csv/'+ticker+'_'+str(year)+'_'+str(month)+'_stocks.csv')

In [22]:
years = [2021,2020,2019,2018]
for ticker in tickers+tickers_done:
    for year in years:
        gen = api.search_comments(q=ticker, 
                              subreddit='stocks',
                              after=int(dt.datetime(year, 12,1).timestamp()),
                              before=int(dt.datetime(year+1, 1,1).timestamp()),
                              filter=['body','author','subreddit']
                             )
        cache=[]
        for i in gen:
            #take only the dictionary from queries
            cache.append(i[-1])
        df = pd.DataFrame(cache)
        if df.empty != True:
            try:
                df['created_utc']=pd.to_datetime(df['created_utc'],unit='s').dt.date
            except KeyError:
                df =df
            df = df.drop(columns=['created'])
            df.rename(columns={'author':'username','created_utc':'date'}, inplace=True)
            df=df[['date','body','username','subreddit']]
            if df.empty != True:
                        df.to_csv('csv/'+ticker+'_'+str(year)+'_'+str(12)+'_stocks.csv')